# 1. Training: load corpus

In [1]:
import pandas as pd
import numpy as np
# from pyabsa.functional import ABSADatasetList
# from pyabsa.functional import ATEPCCheckpointManager
import pickle

In [2]:
res_reviews = pd.read_csv("Restaurant_Reviews.tsv", sep='\t')
review_list = res_reviews.Review.tolist()

In [3]:
try:
    with open('aspect_extractor.pkl', 'rb') as file:
        aspect_extractor = pickle.load(file)


except: 
    print("FAILED!!!")
#     aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english')
#     with open('aspect_extractor.pkl', 'wb') as file:
#         pickle.dump(aspect_extractor, file)

This script could only be used to manage NVIDIA GPUs,but no GPU found in your device


/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/neural/train.py:7: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from .optimizers import Adam, linear_decay
/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/thinc/check.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, Sized, Iterable, Callable
/Users/xiaoxiangma/.local/lib/python3.7/site-packages/matplotlib/backend_bases.py:55: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in Pillow 9 (2022-01-02). Use __version__ instead.
  from PIL import PILLOW_VERSION


Remote ABSADataset version: 2022.07.22 Local ABSADatasets version: None
Unknown local version for ABSADatasets, please check the latest version of ABSADatasets at https://github.com/yangheng95/ABSADatasets


In [4]:
# atepc_result = aspect_extractor.extract_aspect(inference_source=res_reviews.Review.tolist(),
#                                                save_result=True,
#                                                print_result=True,  # print the result
#                                                pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
#                                                )

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 2. Prepare for clustering

In [5]:
import json
  
# Opening JSON file
f = open('atepc_inference.result.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
f.close()

In [6]:
# main = pd.read_json('atepc_inference.result.json')
# main

In [7]:
main = []
for e, sentence in enumerate(data[:]):
    single = []
    if True:
        single.append(e)
        single.append(sentence['sentence'])
        single.append(sentence['tokens'])
        single.append(sentence['aspect'])
        single.append([i[0] for i in sentence['position']])
        single.append(sentence['sentiment'])
        main.append(single)
main = pd.DataFrame(main, columns =['sentence_id', 'sentence', 'tokens', 'aspect', 'aspect_loc', 'aspect_sentiment'])

In [8]:
main

,sentence_id,sentence,tokens,aspect,aspect_loc,aspect_sentiment
0,0,Wow . . . Loved this place .,"[Wow, ., ., ., Loved, this, place, .]",[place],[7],[Positive]
1,1,Crust is not good .,"[Crust, is, not, good, .]",[Crust],[1],[Negative]
2,2,Not tasty and the texture was just nasty .,"[Not, tasty, and, the, texture, was, just, nas...",[texture],[5],[Negative]
3,3,Stopped by during the late May bank holiday of...,"[Stopped, by, during, the, late, May, bank, ho...",[],[],[]
4,4,The selection on the menu was great and so wer...,"[The, selection, on, the, menu, was, great, an...","[selection, menu, prices]","[2, 5, 12]","[Neutral, Neutral, Positive]"
...,...,...,...,...,...,...
995,995,I think food should have flavor and texture an...,"[I, think, food, should, have, flavor, and, te...","[food, flavor, texture]","[3, 6, 8]","[Neutral, Negative, Negative]"
996,996,Appetite instantly gone .,"[Appetite, instantly, gone, .]",[Appetite],[1],[Negative]
997,997,Overall I was not impressed and would not go b...,"[Overall, I, was, not, impressed, and, would, ...",[],[],[]
998,998,"The whole experience was underwhelming , and I...","[The, whole, experience, was, underwhelming, ,...","[experience, Sushi]","[3, 17]","[Negative, Neutral]"


In [9]:
aspects_loc = []
for e, sentence in enumerate(data[:]):
    if sentence['aspect'] != []:
        for i in range(len(sentence['aspect'])):
            aspects_loc.append([e,sentence['position'][i][0],sentence['aspect'][i]])
aspects_loc = pd.DataFrame(aspects_loc, columns =['sentence_id', 'loc','aspect'])


/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
aspects_loc

,sentence_id,loc,aspect
0,0,7,place
1,1,1,Crust
2,2,5,texture
3,4,2,selection
4,4,5,menu
...,...,...,...
1183,995,8,texture
1184,996,1,Appetite
1185,998,3,experience
1186,998,17,Sushi


# 3. Clustering: Umap & HDBscan

In [11]:
import random
from functools import partial

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hdbscan
import umap
import tensorflow as tf

import tensorflow_hub as hub

from sentence_transformers import SentenceTransformer

from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

from tqdm.notebook import trange
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval, Trials

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/xiaoxiangma/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C hea

In [12]:
all_intents = list(aspects_loc['aspect'])
len(all_intents)

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1188

In [13]:
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
# model_use = hub.load(module_url)

# print(f"module {module_url} loaded")

In [14]:
model_st1 = SentenceTransformer('all-mpnet-base-v2')
model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
model_st3 = SentenceTransformer('all-distilroberta-v1')

In [15]:
type(model_st1)

sentence_transformers.SentenceTransformer.SentenceTransformer

In [16]:
with open('POC_sentence_transformer.pkl', 'wb') as file:
    pickle.dump(model_st1, file)

### Embed phrases

In [17]:
def embed(model, model_type, sentences):
    if model_type == 'use':
        embeddings = model(sentences)
    elif model_type == 'sentence transformer':
        embeddings = model.encode(sentences, show_progress_bar=True)
    
    return embeddings

In [18]:
embeddings_st1 = embed(model_st1, 'sentence transformer', all_intents)

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [19]:
# embeddings_st2 = embed(model_st2, 'sentence transformer', all_intents)

In [20]:
# embeddings_st3 = embed(model_st3, 'sentence transformer', all_intents)

In [21]:
# embeddings = [embeddings_st1, embeddings_st2, embeddings_st3]

# for embedding in embeddings:
#     print(embedding.shape)

In [22]:
embeddings_st1.shape

(1188, 768)

### Dimensionality reduction

In [23]:
def umap_fit(message_embeddings,
                      n_neighbors,
                      n_components, 
                      random_state = None):
    reducer = umap.UMAP(n_neighbors = n_neighbors, 
                                n_components = n_components, 
                                metric = 'cosine', 
                                random_state=random_state).fit(message_embeddings)
    return(reducer)

In [27]:
reducer = umap_fit(embeddings_st1, n_neighbors = 15, 
                                     n_components = 5,)

In [28]:
def generate_clusters(umap_reducer,
                      message_embeddings,
                      min_cluster_size,
                      min_samples = None):
    """
    Returns HDBSCAN objects after first performing dimensionality reduction using UMAP
    
    Arguments:
        message_embeddings: embeddings to use
        n_neighbors: int, UMAP hyperparameter n_neighbors
        n_components: int, UMAP hyperparameter n_components
        min_cluster_size: int, HDBSCAN hyperparameter min_cluster_size
        min_samples: int, HDBSCAN hyperparameter min_samples
        random_state: int, random seed
        
    Returns:
        clusters: HDBSCAN object of clusters
    """
    

    umap_embeddings = umap_reducer.transform(message_embeddings)
    
    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size, 
                               min_samples = min_samples,
                               metric='euclidean', 
                               gen_min_span_tree=True,
                               cluster_selection_method='eom',
                              prediction_data=True).fit(umap_embeddings)
    
    return clusters

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
clusters_default = generate_clusters(reducer, embeddings_st1, 
                                     min_cluster_size = 10,)

In [30]:
clusters_default

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
        approx_min_span_tree=True, cluster_selection_epsilon=0.0,
        cluster_selection_method='eom', core_dist_n_jobs=4,
        gen_min_span_tree=True, leaf_size=40,
        match_reference_implementation=False, max_cluster_size=0,
        memory=Memory(location=None), metric='euclidean', min_cluster_size=10,
        min_samples=None, p=None, prediction_data=True)

In [31]:
def score_clusters(clusters, prob_threshold = 0.05):
    """
    Returns the label count and cost of a given clustering

    Arguments:
        clusters: HDBSCAN clustering object
        prob_threshold: float, probability threshold to use for deciding
                        what cluster labels are considered low confidence

    Returns:
        label_count: int, number of unique cluster labels, including noise
        cost: float, fraction of data points whose cluster assignment has
              a probability below cutoff threshold
    """
    
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)
    
    return label_count, cost

In [32]:
score_clusters(clusters_default)

(40, 0.0968013468013468)

In [33]:
def objective(params, embeddings, label_lower, label_upper):
    """
    Objective function for hyperopt to minimize

    Arguments:
        params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'random_state' and
               their values to use for evaluation
        embeddings: embeddings to use
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters

    Returns:
        loss: cost function result incorporating penalties for falling
              outside desired range for number of clusters
        label_count: int, number of unique cluster labels, including noise
        status: string, hypoeropt status

        """
    reducer = umap_fit(embeddings, n_neighbors = params['n_neighbors'], 
                                 n_components = params['n_components'],
                                 random_state = params['random_state'])
    
    clusters = generate_clusters(reducer, 
                                 embeddings,
                                 min_cluster_size = params['min_cluster_size'])
    
    label_count, cost = score_clusters(clusters, prob_threshold = 0.05)
    
    #15% penalty on the cost function if outside the desired range of groups
    if (label_count < label_lower) | (label_count > label_upper):
        penalty = 0.15 
    else:
        penalty = 0
    
    loss = cost + penalty
    
    return {'loss': loss, 'label_count': label_count, 'status': STATUS_OK, 'reducer': reducer, 'HDBSCANclusters':clusters}

In [34]:
objective({
    "n_neighbors": 11,
    "n_components": 8,
    "min_cluster_size": 6,
    "random_state": 42}, embeddings_st1,  label_lower = 15, label_upper = 20
          
         )

{'loss': 0.2106060606060606,
 'label_count': 56,
 'status': 'ok',
 'reducer': UMAP(angular_rp_forest=True, metric='cosine', n_components=8, n_neighbors=11, random_state=42, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'HDBSCANclusters': HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
         approx_min_span_tree=True, cluster_selection_epsilon=0.0,
         cluster_selection_method='eom', core_dist_n_jobs=4,
         gen_min_span_tree=True, leaf_size=40,
         match_reference_implementation=False, max_cluster_size=0,
         memory=Memory(location=None), metric='euclidean', min_cluster_size=6,
         min_samples=None, p=None, prediction_data=True)}

In [35]:
def bayesian_search(embeddings, space, label_lower, label_upper, max_evals=100):
    """
    Perform bayesian search on hyperparameter space using hyperopt

    Arguments:
        embeddings: embeddings to use
        space: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', and 'random_state' and
               values that use built-in hyperopt functions to define
               search spaces for each
        label_lower: int, lower end of range of number of expected clusters
        label_upper: int, upper end of range of number of expected clusters
        max_evals: int, maximum number of parameter combinations to try

    Saves the following to instance variables:
        best_params: dict, contains keys for 'n_neighbors', 'n_components',
               'min_cluster_size', 'min_samples', and 'random_state' and
               values associated with lowest cost scenario tested
        best_clusters: HDBSCAN object associated with lowest cost scenario
                       tested
        trials: hyperopt trials object for search

        """
    
    trials = Trials()
    fmin_objective = partial(objective, 
                             embeddings=embeddings, 
                             label_lower=label_lower,
                             label_upper=label_upper)
    
    best = fmin(fmin_objective, 
                space = space, 
                algo=tpe.suggest,
                max_evals=max_evals, 
                trials=trials)

    best_params = space_eval(space, best)
    print ('best:')
    print (best_params)
    print (f"label count: {trials.best_trial['result']['label_count']}")
    

    best_reducer = umap_fit(embeddings, n_neighbors = best_params['n_neighbors'], 
                            n_components = best_params['n_components'],
                            random_state = best_params['random_state'])
    
    best_clusters = generate_clusters(best_reducer, 
                                      embeddings,
                                      min_cluster_size = best_params['min_cluster_size'])
    
    
    return best_params, best_reducer, best_clusters, trials

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
hspace = {
    "n_neighbors": hp.choice('n_neighbors', range(3,50)),
    "n_components": hp.choice('n_components', range(10,50)),
    "min_cluster_size": hp.choice('min_cluster_size', range(2,16)),
    "random_state": 42
}

label_lower = 10
label_upper = 20
max_evals = 10

In [37]:
best_params_st1, best_reducer_st1, best_clusters_st1, trials_st1 = bayesian_search(embeddings_st1, 
                                                                 space=hspace, 
                                                                 label_lower=label_lower, 
                                                                 label_upper=label_upper, 
                                                                 max_evals=max_evals)

100%|██████████| 10/10 [02:47<00:00, 16.77s/trial, best loss: 0.21228956228956228]
best:
{'min_cluster_size': 4, 'n_components': 41, 'n_neighbors': 14, 'random_state': 42}
label count: 68


In [40]:
# {'min_cluster_size': 4,
#  'n_components': 41,
#  'n_neighbors': 14,
#  'random_state': 42}

In [41]:
with open('POC_best_reducer_st1.pkl', 'wb') as file:
    pickle.dump(best_reducer_st1, file)

In [42]:
with open('POC_best_clusters_st1.pkl', 'wb') as file:
    pickle.dump(best_clusters_st1, file)

In [43]:
# best_params_st2, best_reducer_st2, best_clusters_st2, trials_st2 = bayesian_search(embeddings_st2, 
#                                                                  space=hspace, 
#                                                                  label_lower=label_lower, 
#                                                                  label_upper=label_upper, 
#                                                                  max_evals=max_evals)

In [44]:
# best_params_st3, best_reducer_st3, best_clusters_st3, trials_st3 = bayesian_search(embeddings_st3, 
#                                                                  space=hspace, 
#                                                                  label_lower=label_lower, 
#                                                                  label_upper=label_upper, 
#                                                                  max_evals=max_evals)

{'min_cluster_size': 14, 'n_components': 24, 'n_neighbors': 46, 'random_state': 42}
label count: 11

In [45]:
def combine_results(df_ground, cluster_dict):
    """
    Returns dataframe of all documents and each model's assigned cluster

    Arguments:
        df_ground: dataframe of original documents with associated ground truth
                   labels
        cluster_dict: dict, keys as column name for specific model and value as
                      best clusters HDBSCAN object

    Returns:
        df_combined: dataframe of all documents with labels from
                     best clusters for each model

    """

    df_combined = df_ground.copy()
    
    for key, value in cluster_dict.items():
        df_combined[key] = value.labels_
    
    return df_combined

In [46]:
cluster_dict = {'label_st1': best_clusters_st1}

results_df = combine_results(aspects_loc, cluster_dict)

results_df

,sentence_id,loc,aspect,label_st1
0,0,7,place,9
1,1,1,Crust,27
2,2,5,texture,39
3,4,2,selection,2
4,4,5,menu,11
...,...,...,...,...
1183,995,8,texture,39
1184,996,1,Appetite,23
1185,998,3,experience,34
1186,998,17,Sushi,52


In [47]:
results_df[results_df.label_st1== 1]

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sentence_id,loc,aspect,label_st1
10,10,1,Service,1
16,17,7,service,1
20,22,1,Service,1
42,37,2,service,1
46,39,1,Service,1
...,...,...,...,...
1133,953,4,service,1
1140,960,16,serving,1
1142,961,18,service,1
1146,968,9,service,1


In [48]:
aspect_clusters = results_df.groupby('label_st1').agg(group=('aspect', ','.join)).reset_index()

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
aspect_clusters

,label_st1,group
0,-1,"cranberry,interior,sever,Hiro,styrofoam,regist..."
1,0,"food,food,food,food,food,food,food,food,food,f..."
2,1,"Service,service,Service,service,Service,servic..."
3,2,"selection,food selection,selection,selection,o..."
4,3,"quality,food quality,quality,quality of food,q..."
...,...,...
63,62,"pasta,pasta,noodles,spaghetti,pasta,pastas,pas..."
64,63,"Lobster Bisque soup,Egg Flower Soups,Albondiga..."
65,64,"red velvet cake,dessert section,chocolate milk..."
66,65,"sandwich,English muffin,bread,pork sandwich,sl..."


In [50]:
aspect_clusters.to_csv("POC_clusters.csv")

/Users/xiaoxiangma/anaconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
